### nicholas chung
### data607
### 10/24/19

## assignment: web APIs

The New York Times web site provides a rich set of APIs, as described here: https://developer.nytimes.com/apis

You’ll need to start by signing up for an API key.

Your task is to choose one of the New York Times APIs, construct an interface in Python to read in the JSON data, and
transform it into a pandas DataFrame.

In [2]:
import config
import datetime
import time
import requests

%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd

In [27]:
# define how far back we want to go in our analysis 
weeks = []
# append dates for each of week of the past 3 years to list
for i in range(0, 365, 7):
    weeks.append((datetime.date.today() - datetime.timedelta(i)).isoformat())
    
print(weeks)

# store API key as "key" for easy reference   
key = config.secret

['2019-10-25', '2019-10-18', '2019-10-11', '2019-10-04', '2019-09-27', '2019-09-20', '2019-09-13', '2019-09-06', '2019-08-30', '2019-08-23', '2019-08-16', '2019-08-09', '2019-08-02', '2019-07-26', '2019-07-19', '2019-07-12', '2019-07-05', '2019-06-28', '2019-06-21', '2019-06-14', '2019-06-07', '2019-05-31', '2019-05-24', '2019-05-17', '2019-05-10', '2019-05-03', '2019-04-26', '2019-04-19', '2019-04-12', '2019-04-05', '2019-03-29', '2019-03-22', '2019-03-15', '2019-03-08', '2019-03-01', '2019-02-22', '2019-02-15', '2019-02-08', '2019-02-01', '2019-01-25', '2019-01-18', '2019-01-11', '2019-01-04', '2018-12-28', '2018-12-21', '2018-12-14', '2018-12-07', '2018-11-30', '2018-11-23', '2018-11-16', '2018-11-09', '2018-11-02', '2018-10-26']


In [3]:
# confirm authentication and successful HTTPS request
r = requests.get('https://api.nytimes.com/svc/books/v3/lists/current/hardcover-fiction.json?api-key=' + key)
r.status_code

200

In [29]:
# store URL as fragments for ease of reuse
url_a = "https://api.nytimes.com/svc/books/v3/" 
url_b = "lists/" 
url_d = "/hardcover-fiction.json?" 

books = []
books_d = {}

# grab values for each object returned, then store as list of dicts
for c in weeks:
    r = requests.get(url_a + url_b + c + url_d + "api-key=" + key)
    time.sleep(6) # rate limit guidance as per https://developer.nytimes.com/faq#a11
    for i in r.json()['results']['books']:
        # create dictionary for each result
        books_d = {
            "date": c,
            "title": i['title'], 
            "rank": i['rank'], 
            "rank_last_week": i['rank_last_week'], 
            "weeks_on_list": i['weeks_on_list'], 
            "description": i['description'], 
            "publisher": i['publisher'], 
            "author": i['author']
        }
        # append dictionary to list
        print(books_d)
        books.append(books_d)

print(books)

{'date': '2019-10-25', 'title': 'THE 19TH CHRISTMAS', 'rank': 1, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'In the 19th installment of the Women\'s Murder Club series, detective Lindsay Boxer and company take on a fearsome criminal known only as "Loman."', 'publisher': 'Little, Brown', 'author': 'James Patterson and Maxine Paetro'}
{'date': '2019-10-25', 'title': 'WHERE THE CRAWDADS SING', 'rank': 2, 'rank_last_week': 3, 'weeks_on_list': 58, 'description': 'A woman who survived alone in the marsh becomes a murder suspect.', 'publisher': 'Putnam', 'author': 'Delia Owens'}
{'date': '2019-10-25', 'title': 'WHAT HAPPENS IN PARADISE', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'In the sequel to "Winter in Paradise," Irene Steele visits the island of St. John to get to the bottom of the mysterious life and death of her husband.', 'publisher': 'Little, Brown', 'author': 'Elin Hilderbrand'}
{'date': '2019-10-25', 'title': 'NINTH HOUSE', 'rank': 4, 'rank_la

{'date': '2019-10-11', 'title': 'THE WATER DANCER', 'rank': 1, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A young man who was gifted with a mysterious power becomes part of a war between slavers and the enslaved.', 'publisher': 'One World', 'author': 'Ta-Nehisi Coates'}
{'date': '2019-10-11', 'title': 'THE INSTITUTE', 'rank': 2, 'rank_last_week': 1, 'weeks_on_list': 3, 'description': 'Children with special talents are abducted and sequestered in an institution where the sinister staff seeks to extract their gifts through harsh methods.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2019-10-11', 'title': 'THE TESTAMENTS', 'rank': 3, 'rank_last_week': 2, 'weeks_on_list': 3, 'description': 'In a sequel to “The Handmaid’s Tale,” old secrets bring three women together as the Republic of Gilead’s theocratic regime shows signs of decay.', 'publisher': 'Nan A. Talese/Doubleday', 'author': 'Margaret Atwood'}
{'date': '2019-10-11', 'title': 'VINCE FLYNN: LETHAL AGEN

{'date': '2019-09-27', 'title': 'THE TESTAMENTS', 'rank': 1, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'In a sequel to “The Handmaid’s Tale,” old secrets bring three women together as the Republic of Gilead’s theocratic regime shows signs of decay.', 'publisher': 'Nan A. Talese/Doubleday', 'author': 'Margaret Atwood'}
{'date': '2019-09-27', 'title': 'THE INSTITUTE', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Children with special talents are abducted and sequestered in an institution where the sinister staff seeks to extract their gifts through harsh methods.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2019-09-27', 'title': 'WHERE THE CRAWDADS SING', 'rank': 3, 'rank_last_week': 1, 'weeks_on_list': 54, 'description': 'A woman who survived alone in the marsh becomes a murder suspect.', 'publisher': 'Putnam', 'author': 'Delia Owens'}
{'date': '2019-09-27', 'title': 'KILLER INSTINCT', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list'

{'date': '2019-09-13', 'title': 'A BETTER MAN', 'rank': 1, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'The 15th book in the Chief Inspector Gamache series. The search for a missing girl is imperiled by rising floodwaters across the province.', 'publisher': 'Minotaur', 'author': 'Louise Penny'}
{'date': '2019-09-13', 'title': 'WHERE THE CRAWDADS SING', 'rank': 2, 'rank_last_week': 1, 'weeks_on_list': 52, 'description': 'A woman who survived alone in the marsh becomes a murder suspect.', 'publisher': 'Putnam', 'author': 'Delia Owens'}
{'date': '2019-09-13', 'title': 'THE GIRL WHO LIVED TWICE', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': "Mikael Blomkvist helps Lisbeth Salander put her past behind her in the latest installment of Stieg Larsson's Millennium series.", 'publisher': 'Knopf', 'author': 'David Lagercrantz'}
{'date': '2019-09-13', 'title': 'THE DARK SIDE', 'rank': 4, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Painful childhood me

{'date': '2019-08-30', 'title': 'WHERE THE CRAWDADS SING', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 50, 'description': 'A woman who survived alone in the marsh becomes a murder suspect.', 'publisher': 'Putnam', 'author': 'Delia Owens'}
{'date': '2019-08-30', 'title': 'THE INN', 'rank': 2, 'rank_last_week': 2, 'weeks_on_list': 2, 'description': 'A former Boston police detective who is now an innkeeper must shield a seaside town from a crew of criminals.', 'publisher': 'Little, Brown', 'author': 'James Patterson and Candice Fox'}
{'date': '2019-08-30', 'title': 'THE BITTERROOTS', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'The fourth book in the Cassie Dewell series. The black sheep of an influential family is accused of assault.', 'publisher': 'Minotaur', 'author': 'CJ Box'}
{'date': '2019-08-30', 'title': 'ONE GOOD DEED', 'rank': 4, 'rank_last_week': 4, 'weeks_on_list': 4, 'description': 'A World War II veteran on parole must find the real killer in a sm

...

{'date': '2018-12-28', 'title': 'THE RECKONING', 'rank': 1, 'rank_last_week': 1, 'weeks_on_list': 8, 'description': 'A decorated World War II veteran shoots and kills a pastor inside a Mississippi church.', 'publisher': 'Doubleday', 'author': 'John Grisham'}
{'date': '2018-12-28', 'title': 'FIRE AND BLOOD', 'rank': 2, 'rank_last_week': 3, 'weeks_on_list': 4, 'description': 'Set 300 years before the events of “A Game of Thrones,” this is the first volume of the two-part history of the Targaryens in Westeros.', 'publisher': 'Bantam', 'author': 'George RR Martin'}
{'date': '2018-12-28', 'title': 'EVERY BREATH', 'rank': 3, 'rank_last_week': 4, 'weeks_on_list': 9, 'description': 'Difficult choices surface when Hope Anderson and Tru Walls meet in a North Carolina seaside town.', 'publisher': 'Grand Central', 'author': 'Nicholas Sparks'}
{'date': '2018-12-28', 'title': 'WHERE THE CRAWDADS SING', 'rank': 4, 'rank_last_week': 11, 'weeks_on_list': 15, 'description': 'A woman who survived alone i

{'date': '2018-12-14', 'title': 'KINGDOM OF THE BLIND', 'rank': 1, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'While on suspension, Chief Inspector Armand Gamache is made an executor of a stranger’s will and tries to keep a deadly narcotic off Montreal’s streets.', 'publisher': 'Minotaur', 'author': 'Louise Penny'}
{'date': '2018-12-14', 'title': 'FIRE AND BLOOD', 'rank': 2, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'Set 300 years before the events of “A Game of Thrones,” this is the first volume of the two-part history of the Targaryens in Westeros.', 'publisher': 'Bantam', 'author': 'George RR Martin'}
{'date': '2018-12-14', 'title': 'THE RECKONING', 'rank': 3, 'rank_last_week': 3, 'weeks_on_list': 6, 'description': 'A decorated World War II veteran shoots and kills a pastor inside a Mississippi church.', 'publisher': 'Doubleday', 'author': 'John Grisham'}
{'date': '2018-12-14', 'title': 'TARGET: ALEX CROSS', 'rank': 4, 'rank_last_week': 2, 'weeks_on_list':

{'date': '2018-11-30', 'title': 'LOOK ALIVE TWENTY-FIVE', 'rank': 1, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'The 25th book in the Stephanie Plum series. When several managers of a deli in Trenton disappear, a bounty hunter and her detective boyfriend look for clues.', 'publisher': 'Putnam', 'author': 'Janet Evanovich'}
{'date': '2018-11-30', 'title': 'LONG ROAD TO MERCY', 'rank': 2, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': "Atlee Pine, an F.B.I. agent who is haunted by the childhood kidnapping of her twin sister, must choose between protecting her career or the United States' democracy.", 'publisher': 'Grand Central', 'author': 'David Baldacci'}
{'date': '2018-11-30', 'title': 'THE RECKONING', 'rank': 3, 'rank_last_week': 3, 'weeks_on_list': 4, 'description': 'A decorated World War II veteran shoots and kills a pastor inside a Mississippi church.', 'publisher': 'Doubleday', 'author': 'John Grisham'}
{'date': '2018-11-30', 'title': 'PAST TENSE', 'rank': 4

{'date': '2018-11-16', 'title': 'ELEVATION', 'rank': 1, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'A man who is losing weight without getting thinner forms an unlikely alliance with his neighbors who are dealing with prejudices.', 'publisher': 'Scribner', 'author': 'Stephen King'}
{'date': '2018-11-16', 'title': 'THE RECKONING', 'rank': 2, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'A decorated World War II veteran shoots and kills a pastor inside a Mississippi church.', 'publisher': 'Doubleday', 'author': 'John Grisham'}
{'date': '2018-11-16', 'title': 'DARK SACRED NIGHT', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Detective Renée Ballard teams up with the retired detective Harry Bosch, who is working on a cold case.', 'publisher': 'Little, Brown', 'author': 'Michael Connelly'}
{'date': '2018-11-16', 'title': 'EVERY BREATH', 'rank': 4, 'rank_last_week': 2, 'weeks_on_list': 3, 'description': 'Difficult choices surface when Hope Anders

{'date': '2018-11-02', 'title': 'EVERY BREATH', 'rank': 1, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Difficult choices surface when Hope Anderson and Tru Walls meet in a North Carolina seaside town.', 'publisher': 'Grand Central', 'author': 'Nicholas Sparks'}
{'date': '2018-11-02', 'title': 'THE NEXT PERSON YOU MEET IN HEAVEN', 'rank': 2, 'rank_last_week': 1, 'weeks_on_list': 2, 'description': 'The sequel to "The Five People You Meet in Heaven" follows Annie on her heavenly journey.', 'publisher': 'Harper', 'author': 'Mitch Albom'}
{'date': '2018-11-02', 'title': 'UNSHELTERED', 'rank': 3, 'rank_last_week': 0, 'weeks_on_list': 1, 'description': 'Intertwined stories of two families who live in different centuries on the same street corner in Vineland, N.J.', 'publisher': 'Harper', 'author': 'Barbara Kingsolver'}
{'date': '2018-11-02', 'title': 'A SPARK OF LIGHT', 'rank': 4, 'rank_last_week': 6, 'weeks_on_list': 3, 'description': "The lives of patients, doctors and activist

In [30]:
# store list of dictionanies to dataframe
books_df = pd.DataFrame(books)
books_df

,date,title,rank,rank_last_week,weeks_on_list,description,publisher,author
0,2019-10-25,THE 19TH CHRISTMAS,1,0,1,In the 19th installment of the Women's Murder ...,"Little, Brown",James Patterson and Maxine Paetro
1,2019-10-25,WHERE THE CRAWDADS SING,2,3,58,A woman who survived alone in the marsh become...,Putnam,Delia Owens
2,2019-10-25,WHAT HAPPENS IN PARADISE,3,0,1,"In the sequel to ""Winter in Paradise,"" Irene S...","Little, Brown",Elin Hilderbrand
3,2019-10-25,NINTH HOUSE,4,0,1,After mysteriously surviving a multiple homici...,Flatiron,Leigh Bardugo
4,2019-10-25,THE INSTITUTE,5,2,5,Children with special talents are abducted and...,Scribner,Stephen King
...,...,...,...,...,...,...,...,...
790,2018-10-26,ALASKAN HOLIDAY,11,4,2,"A Seattleite cook stuck in remote Ponder, Alas...",Ballantine,Debbie Macomber
791,2018-10-26,LETHAL WHITE,12,7,4,The fourth book in the Cormoran Strike series....,"Mulholland/Little, Brown",Robert Galbraith
792,2018-10-26,JUROR #3,13,6,5,Ruby Bozarth defends a college football star c...,"Little, Brown",James Patterson and Nancy Allen
793,2018-10-26,AN ABSOLUTELY REMARKABLE THING,14,3,3,"After posting a video that goes viral, April M...",Dutton,Hank Green


In [31]:
books_df.to_csv("books.csv")

In [32]:
books_df = pd.read_csv("books.csv", index_col = 0)
books_df

,date,title,rank,rank_last_week,weeks_on_list,description,publisher,author
0,2019-10-25,THE 19TH CHRISTMAS,1,0,1,In the 19th installment of the Women's Murder ...,"Little, Brown",James Patterson and Maxine Paetro
1,2019-10-25,WHERE THE CRAWDADS SING,2,3,58,A woman who survived alone in the marsh become...,Putnam,Delia Owens
2,2019-10-25,WHAT HAPPENS IN PARADISE,3,0,1,"In the sequel to ""Winter in Paradise,"" Irene S...","Little, Brown",Elin Hilderbrand
3,2019-10-25,NINTH HOUSE,4,0,1,After mysteriously surviving a multiple homici...,Flatiron,Leigh Bardugo
4,2019-10-25,THE INSTITUTE,5,2,5,Children with special talents are abducted and...,Scribner,Stephen King
...,...,...,...,...,...,...,...,...
790,2018-10-26,ALASKAN HOLIDAY,11,4,2,"A Seattleite cook stuck in remote Ponder, Alas...",Ballantine,Debbie Macomber
791,2018-10-26,LETHAL WHITE,12,7,4,The fourth book in the Cormoran Strike series....,"Mulholland/Little, Brown",Robert Galbraith
792,2018-10-26,JUROR #3,13,6,5,Ruby Bozarth defends a college football star c...,"Little, Brown",James Patterson and Nancy Allen
793,2018-10-26,AN ABSOLUTELY REMARKABLE THING,14,3,3,"After posting a video that goes viral, April M...",Dutton,Hank Green


In [33]:
books_df.loc[books_df['rank'] == 5]

,date,title,rank,rank_last_week,weeks_on_list,description,publisher,author
4,2019-10-25,THE INSTITUTE,5,2,5,Children with special talents are abducted and...,Scribner,Stephen King
19,2019-10-18,BLOODY GENIUS,5,0,1,The 12th book in the Virgil Flowers series. A ...,Putnam,John Sandford
34,2019-10-11,WHERE THE CRAWDADS SING,5,3,56,A woman who survived alone in the marsh become...,Putnam,Delia Owens
49,2019-10-04,THE GIRL WHO LIVED TWICE,5,6,4,Mikael Blomkvist helps Lisbeth Salander put he...,Knopf,David Lagercrantz
64,2019-09-27,THE ORACLE,5,4,2,A traveler discovers mysteries hidden behind s...,Front Line,Jonathan Cahn
79,2019-09-20,A BETTER MAN,5,1,2,The 15th book in the Chief Inspector Gamache s...,Minotaur,Louise Penny
94,2019-09-13,THE INN,5,3,4,A former Boston police detective who is now an...,"Little, Brown",James Patterson and Candice Fox
109,2019-09-06,THE NICKEL BOYS,5,5,6,Two boys respond to horrors at a Jim Crow-era ...,Doubleday,Colson Whitehead
124,2019-08-30,THE NICKEL BOYS,5,6,5,Two boys respond to horrors at a Jim Crow-era ...,Doubleday,Colson Whitehead
139,2019-08-23,OUTFOX,5,0,1,F.B.I. Agent Drex Easton has a hunch that the ...,Grand Central,Sandra Brown


In [34]:
books_df.title.unique()

array(['THE 19TH CHRISTMAS', 'WHERE THE CRAWDADS SING',
       'WHAT HAPPENS IN PARADISE', 'NINTH HOUSE', 'THE INSTITUTE',
       'THE WATER DANCER', 'THE DUTCH HOUSE', 'THE TESTAMENTS',
       "CHILD'S PLAY", 'THE GIVER OF STARS', 'BLOODY GENIUS',
       'VINCE FLYNN: LETHAL AGENT', "CILKA'S JOURNEY", 'IMAGINARY FRIEND',
       'THE ORACLE', 'FULL THROTTLE', 'A MRS. MIRACLE CHRISTMAS',
       'THE GIRL WHO LIVED TWICE', 'RED AT THE BONE',
       'THE SECRETS WE KEPT', 'KILLER INSTINCT', 'THE NICKEL BOYS',
       'THE WORLD THAT WE KNEW', 'THE TITANIC SECRET', 'A BETTER MAN',
       'LAND OF WOLVES', 'VENDETTA IN DEATH', 'THIS TENDER LAND',
       'A LITTLE HATRED', 'THE DARK SIDE', 'ONE GOOD DEED', 'QUICHOTTE',
       'THE INN', 'THE TURN OF THE KEY', 'DARK ILLUSION', 'OLD BONES',
       'THE SILENT PATIENT', 'ASK AGAIN, YES', 'THE BITTERROOTS',
       'INLAND', 'OUTFOX', 'THE NEW GIRL', 'THE LAST WIDOW',
       'THE WHISPER MAN', 'TIDELANDS', 'EVVIE DRAKE STARTS OVER',
       'CHANCE

In [24]:
r = requests.get('https://api.nytimes.com/svc/books/v3/lists/2016-11-10/hardcover-fiction.json?api-key=' + key)
r.json()['results']['books']

[{'rank': 1,
  'rank_last_week': 0,
  'weeks_on_list': 1,
  'asterisk': 0,
  'dagger': 0,
  'primary_isbn10': '0385541198',
  'primary_isbn13': '9780385541190',
  'publisher': 'Doubleday',
  'description': 'A whistleblower alerts a Florida investigator to judicial corruption involving the Mob and Indian casinos.',
  'price': 0,
  'title': 'THE WHISTLER',
  'author': 'John Grisham',
  'contributor': 'by John Grisham',
  'contributor_note': '',
  'book_image': 'https://s1.nyt.com/du/books/images/9780385541206.jpg',
  'book_image_width': 326,
  'book_image_height': 495,
  'amazon_product_url': 'https://www.amazon.com/Whistler-John-Grisham-ebook/dp/B01C1LUFFK?tag=NYTBS-20',
  'age_group': '',
  'book_review_link': 'https://www.nytimes.com/2016/11/06/books/review/john-grisham-whistler.html',
  'first_chapter_link': '',
  'sunday_review_link': '',
  'article_chapter_link': '',
  'isbns': [{'isbn10': '0385541198', 'isbn13': '9780385541190'},
   {'isbn10': '0385541201', 'isbn13': '978038554120